## Punto 1 - Planeación

### A. Reconocimiento y tratamiento de atributos con valores únicos o distintos

#### Hallazgos:
- **PassengerId**: Variable con valores únicos para cada registro.
- **Name**: Variable con valores distintos para cada registro, pero contiene títulos que podrían ser útiles.

#### Propuestas:
- Descartar "PassengerId".
- Extraer títulos de "Name" antes de descartarla.

### B. Reconocimiento y tratamiento de atributos con valores faltantes

#### Hallazgos:
- **Age**, **Cabin**, y **Embarked** tienen valores faltantes.

#### Propuestas:
- Imputar los valores faltantes de "Age" con la mediana.
- Imputar "Embarked" con el puerto más común.
- Descartar "Cabin" debido a la gran cantidad de valores faltantes.

### C. Reconocimiento y tratamiento de atributos con valores atípicos

#### Hallazgos:
- **Age** y **Fare** podrían contener valores atípicos.

#### Propuestas:
- Utilizar el método IQR para detectar y tratar los valores atípicos en "Age" y "Fare".

### D. Reconocimiento y tratamiento de registros atípicos

#### Hallazgos:
- Algunos registros podrían contener combinaciones de variables que son atípicas o inusuales.

#### Propuestas:
- Investigar y validar si esos registros son errores o outliers naturales, y decidir si se mantienen o se descartan.

### E. Reconocimiento y tratamiento de atributos redundantes o que no dan valor

#### Hallazgos:
- **Ticket** y **Cabin** podrían no aportar valor al modelo.

#### Propuestas:
- Validar la utilidad de estas variables y decidir si se mantienen, transforman o descartan.


In [4]:
import pandas as pd

# Cargar el dataset
url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
df_titanic = pd.read_csv(url)

# A. Tratamiento de valores únicos
df_titanic['Title'] = df_titanic['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())
df_titanic.drop(['PassengerId', 'Name'], axis=1, inplace=True)

# B. Tratamiento de valores faltantes
df_titanic['Age'].fillna(df_titanic['Age'].median(), inplace=True)
df_titanic['Embarked'].fillna(df_titanic['Embarked'].mode()[0], inplace=True)
df_titanic.drop(['Cabin'], axis=1, inplace=True)

# C. Tratamiento de valores atípicos
# --- AGE ---

# Calculando IQR para 'Age'
Q1_age = df_titanic['Age'].quantile(0.25)
Q3_age = df_titanic['Age'].quantile(0.75)
IQR_age = Q3_age - Q1_age

# Calculando límites para outliers en 'Age'
lower_bound_age = Q1_age - 1.5 * IQR_age
upper_bound_age = Q3_age + 1.5 * IQR_age

# Identificando outliers en 'Age'
outliers_age = df_titanic[(df_titanic['Age'] < lower_bound_age) | (df_titanic['Age'] > upper_bound_age)]

# Tratamiento de outliers en 'Age'
# Opción 1: Eliminar los registros con outliers en 'Age'
df_titanic = df_titanic[(df_titanic['Age'] >= lower_bound_age) & (df_titanic['Age'] <= upper_bound_age)]

# --- FARE ---

# Calculando IQR para 'Fare'
Q1_fare = df_titanic['Fare'].quantile(0.25)
Q3_fare = df_titanic['Fare'].quantile(0.75)
IQR_fare = Q3_fare - Q1_fare

# Calculando límites para outliers en 'Fare'
lower_bound_fare = Q1_fare - 1.5 * IQR_fare
upper_bound_fare = Q3_fare + 1.5 * IQR_fare

# Identificando outliers en 'Fare'
outliers_fare = df_titanic[(df_titanic['Fare'] < lower_bound_fare) | (df_titanic['Fare'] > upper_bound_fare)]

# Tratamiento de outliers en 'Fare'
# Opción 2: Imputar los outliers con la mediana de 'Fare'
median_fare = df_titanic['Fare'].median()
df_titanic.loc[(df_titanic['Fare'] < lower_bound_fare) | (df_titanic['Fare'] > upper_bound_fare), 'Fare'] = median_fare

# Resultados de la limpieza
print(f"Número de outliers identificados en 'Age': {len(outliers_age)}")
print(f"Número de outliers identificados en 'Fare': {len(outliers_fare)}")
print(f"Dimensiones del DataFrame después de la limpieza: {df_titanic.shape}")

# D. Tratamiento de registros atípicos (requiere análisis específico)
# Identificación de registros atípicos
outliers = df_titanic[df_titanic['Age'] > 100]

# Tratamiento de registros atípicos
# Opción 1: Eliminar los registros atípicos
df_titanic = df_titanic[df_titanic['Age'] <= 100]

# Opción 2: Imputar los registros atípicos con un valor específico (por ejemplo, la mediana de 'Age')
# df_titanic.loc[df_titanic['Age'] > 100, 'Age'] = df_titanic['Age'].median()


# E. Tratamiento de atributos que no dan valor
df_titanic.drop(['Ticket'], axis=1, inplace=True)

# Guardar el dataframe limpio
df_titanic.to_csv('titanic_cleaned_def.csv', index=False)


Número de outliers identificados en 'Age': 66
Número de outliers identificados en 'Fare': 107
Dimensiones del DataFrame después de la limpieza: (825, 10)


In [5]:
# archivo limpio
titanic_df = pd.read_csv("/content/titanic_cleaned_def.csv")
titanic_df.head(100)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,male,22.0,1,0,7.2500,S,Mr
1,1,1,female,38.0,1,0,13.4167,C,Mrs
2,1,3,female,26.0,0,0,7.9250,S,Miss
3,1,1,female,35.0,1,0,53.1000,S,Mrs
4,0,3,male,35.0,0,0,8.0500,S,Mr
...,...,...,...,...,...,...,...,...,...
95,0,3,male,37.0,2,0,7.9250,S,Mr
96,0,3,male,28.0,0,0,7.8958,S,Mr
97,1,3,female,21.0,0,0,7.6500,S,Miss
98,1,3,male,28.0,0,0,7.7750,S,Mr
